In [1]:
import os
%pwd

'c:\\Users\\hp\\Documents\\DS\\Complete Project\\04-Mobile-Price-Predictor\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\hp\\Documents\\DS\\Complete Project\\04-Mobile-Price-Predictor'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [16]:
from Mobile_Price_Predictor.constants import *
from Mobile_Price_Predictor.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [18]:
import os
import urllib.request as request
import zipfile
from Mobile_Price_Predictor.logging import logger

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")  

        
    
    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-01 23:05:07,214: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-01 23:05:07,219: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-01 23:05:07,222: INFO: common: created directory at: artifacts]
[2024-05-01 23:05:07,225: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-01 23:05:08,590: INFO: 2283359830: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 69765
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f205e9c80b9eb43acd670ef6cccc3400c58e33b0dafba17f3c41ce210a3490f5"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 8400:B3CBD:EEA5A:13AB69:66327D4A
Accept-Ranges: bytes
Date: Wed, 01 May 2024 17:35:08 GMT
Via: 1.1 varnish
X-Served-By: cache-del21745-DEL
X-Cache: MISS
